In [1]:
#  Required Install
!pip install librosa

In [2]:
#Import Dependencies
from pylab import *
import os
import pandas as pd
import librosa
import librosa.display
import glob
import random

# Sample one audio File

In [3]:
data, sampling_rate = librosa.load('../data/emotion_speech/03-01-08-01-02-02-02.wav')
plt.figure(figsize=(12, 4))

print(data, sampling_rate)
librosa.display.waveplot(data, sr=sampling_rate)

[0. 0. 0. ... 0. 0. 0.] 22050


# RAVDASS AUDIO SET


# Generate list of file and apply emotion str

In [4]:
##Get list of Files in folder
file_name = []

for root, dirs, files in os.walk("../data/emotion_speech/."):
    for filename in files:
        file_name.append(filename)

file_name_df = pd.DataFrame(file_name, columns = ['file_name'])
file_name_df['encoding']= file_name_df['file_name']
file_name_df['ID'] = file_name_df.index 
file_name_df['dir']= 'emotion_speech'
file_name_df

,file_name,encoding,ID,dir
0,03-01-01-01-01-01-01.wav,03-01-01-01-01-01-01.wav,0,emotion_speech
1,03-01-01-01-01-01-02.wav,03-01-01-01-01-01-02.wav,1,emotion_speech
2,03-01-01-01-01-01-03.wav,03-01-01-01-01-01-03.wav,2,emotion_speech
3,03-01-01-01-01-01-04.wav,03-01-01-01-01-01-04.wav,3,emotion_speech
4,03-01-01-01-01-01-05.wav,03-01-01-01-01-01-05.wav,4,emotion_speech
...,...,...,...,...
1435,03-01-08-02-02-02-20.wav,03-01-08-02-02-02-20.wav,1435,emotion_speech
1436,03-01-08-02-02-02-21.wav,03-01-08-02-02-02-21.wav,1436,emotion_speech
1437,03-01-08-02-02-02-22.wav,03-01-08-02-02-02-22.wav,1437,emotion_speech
1438,03-01-08-02-02-02-23.wav,03-01-08-02-02-02-23.wav,1438,emotion_speech


In [73]:
#Create DF seperating Values
# Filename identifiers

# Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
# Vocal channel (01 = speech, 02 = song).
# Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
# Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
# Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
# Repetition (01 = 1st repetition, 02 = 2nd repetition).
# Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

encoding_df = file_name_df["encoding"].str.split("-", n=-1, expand=True)
encoding_df.columns = ['Modality','Vocal_channel','Class','Intensity','Statement','Repetition','Actor']

"""
1. Added a new column - Gender
2. Populated Gender column after stripping .wav from 'Actor'
3. Convert data type from obj to string then to int which is 
   needed to determine odd or even
"""
encoding_df['Actor'] = encoding_df['Actor'].map(lambda actor: actor.rstrip('.wav'))
encoding_df['Gender'] = 'Gender'
encoding_df['Actor'].astype(str).astype(int)

encoding_df
"""
1. Loop through each row in dataframe 'Actor' column
2. Set value of Gender to Female if Actor value is even or Male if odd
"""
for index, row in encoding_df.iterrows():
    if int(row[6]) % 2 == 0:
        row['Gender'] = 'Female'
    else:
        row['Gender'] = 'Male'
        
encoding_df

,Modality,Vocal_channel,Class,Intensity,Statement,Repetition,Actor,Gender
0,03,01,01,01,01,01,01,Male
1,03,01,01,01,01,01,02,Female
2,03,01,01,01,01,01,03,Male
3,03,01,01,01,01,01,04,Female
4,03,01,01,01,01,01,05,Male
...,...,...,...,...,...,...,...,...
1435,03,01,08,02,02,02,20,Female
1436,03,01,08,02,02,02,21,Male
1437,03,01,08,02,02,02,22,Female
1438,03,01,08,02,02,02,23,Male


In [74]:
final_df = encoding_df.drop(columns=['Modality','Vocal_channel','Intensity','Statement','Repetition','Actor'])
final_df.loc[final_df.Class == '01', 'Class'] = '02'
final_df.loc[final_df.Class == '02', 'Emotion'] = 'calm'
final_df.loc[final_df.Class == '03', 'Emotion'] = 'happy'
final_df.loc[final_df.Class == '04', 'Emotion'] = 'sad'
final_df.loc[final_df.Class == '05', 'Emotion'] = 'angry'
final_df.loc[final_df.Class == '06', 'Emotion'] = 'fearful'
final_df.loc[final_df.Class == '07', 'Emotion'] = 'disgust'
final_df.loc[final_df.Class == '08', 'Emotion'] = 'surprise'

final_df
#merge data frame
pre_merged_df = file_name_df.join(final_df, how='outer')
merged_df = pre_merged_df.drop(columns=['encoding'])
merged_df.set_index('ID')

,file_name,dir,Class,Gender,Emotion
ID,,,,,
0,03-01-01-01-01-01-01.wav,emotion_speech,02,Male,calm
1,03-01-01-01-01-01-02.wav,emotion_speech,02,Female,calm
2,03-01-01-01-01-01-03.wav,emotion_speech,02,Male,calm
3,03-01-01-01-01-01-04.wav,emotion_speech,02,Female,calm
4,03-01-01-01-01-01-05.wav,emotion_speech,02,Male,calm
...,...,...,...,...,...
1435,03-01-08-02-02-02-20.wav,emotion_speech,08,Female,surprise
1436,03-01-08-02-02-02-21.wav,emotion_speech,08,Male,surprise
1437,03-01-08-02-02-02-22.wav,emotion_speech,08,Female,surprise


In [109]:
merged_df.Class.value_counts()

02    288
04    192
08    192
05    192
07    192
06    192
03    192
Name: Class, dtype: int64

# TORONTO EMOTIONAL SPEECH DATASET

In [110]:
##Get list of Files in folder
t_file_name = []

for root, dirs, files in os.walk("../data/toronto_speech/."):
    for filename in files:
        t_file_name.append(filename)

t_file_name_df = pd.DataFrame(t_file_name, columns = ['file_name'])
t_file_name_df['encoding']= t_file_name_df['file_name']
t_file_name_df['ID'] = t_file_name_df.index 
t_file_name_df['dir'] = 'toronto_speech'
t_file_name_df.head(20)

,file_name,encoding,ID,dir
0,OAF_back_angry.wav,OAF_back_angry.wav,0,toronto_speech
1,OAF_back_disgust.wav,OAF_back_disgust.wav,1,toronto_speech
2,OAF_back_fear.wav,OAF_back_fear.wav,2,toronto_speech
3,OAF_back_happy.wav,OAF_back_happy.wav,3,toronto_speech
4,OAF_back_neutral.wav,OAF_back_neutral.wav,4,toronto_speech
5,OAF_back_ps.wav,OAF_back_ps.wav,5,toronto_speech
6,OAF_back_sad.wav,OAF_back_sad.wav,6,toronto_speech
7,OAF_bar_angry.wav,OAF_bar_angry.wav,7,toronto_speech
8,OAF_bar_disgust.wav,OAF_bar_disgust.wav,8,toronto_speech
9,OAF_bar_fear.wav,OAF_bar_fear.wav,9,toronto_speech


In [111]:
t_encoding_df = t_file_name_df["encoding"].str.split("_", n=-1, expand=True)
t_encoding_df.columns = ['Actor','Word','Emotion']
#t_encoding_df
t_final_df = t_encoding_df.drop(columns=['Actor','Word'])
t_final_df = t_final_df['Emotion'].str.strip('.wav')
toronto_df = pd.DataFrame(t_final_df)
toronto_df

,Emotion
0,ngry
1,disgust
2,fear
3,happy
4,neutral
...,...
2795,fear
2796,happy
2797,neutral
2798,ps


In [112]:
toronto_df.loc[toronto_df.Emotion == 'neutral', 'Class'] = '02'
toronto_df.loc[toronto_df.Emotion == 'alm', 'Class'] = '02'
toronto_df.loc[toronto_df.Emotion == 'happy', 'Class'] = '03'
toronto_df.loc[toronto_df.Emotion == 'sad', 'Class'] = '04'
toronto_df.loc[toronto_df.Emotion == 'ngry', 'Class'] = '05'
toronto_df.loc[toronto_df.Emotion == 'fear', 'Class'] = '06'
toronto_df.loc[toronto_df.Emotion == 'disgust', 'Class'] = '07'
toronto_df.loc[toronto_df.Emotion == 'ps', 'Class'] = '08'

toronto_df.head(15)

,Emotion,Class
0,ngry,05
1,disgust,07
2,fear,06
3,happy,03
4,neutral,02
5,ps,08
6,sad,04
7,ngry,05
8,disgust,07
9,fear,06


In [92]:
#merge data frame and export CSV
t_pre_merged_df = t_file_name_df.join(toronto_df, how='outer')
t_merged_df = t_pre_merged_df.drop(columns=['encoding'])
t_merged_df = t_merged_df[['file_name', 'dir', 'Class','Emotion','ID']]
t_merged_df
t_merged_df.set_index('ID')

,file_name,dir,Class,Emotion
ID,,,,
0,OAF_back_angry.wav,toronto_speech,05,ngry
1,OAF_back_disgust.wav,toronto_speech,07,disgust
2,OAF_back_fear.wav,toronto_speech,06,fear
3,OAF_back_happy.wav,toronto_speech,03,happy
4,OAF_back_neutral.wav,toronto_speech,02,neutral
...,...,...,...,...
2795,YAF_youth_fear.wav,toronto_speech,06,fear
2796,YAF_youth_happy.wav,toronto_speech,03,happy
2797,YAF_youth_neutral.wav,toronto_speech,02,neutral


In [93]:
t_merged_df.Class.value_counts()

04    400
03    400
08    400
02    400
05    400
07    400
06    400
Name: Class, dtype: int64

# CONCAT DATASETS & RESET INDEX

In [113]:
dataset_df = pd.concat([merged_df, t_merged_df])
dataset2 = dataset_df.reset_index(drop=True)
dataset2['ID']= dataset2.index
final_dataset_df = dataset2
final_dataset_df

,file_name,ID,dir,Class,Gender,Emotion
0,03-01-01-01-01-01-01.wav,0,emotion_speech,02,Male,calm
1,03-01-01-01-01-01-02.wav,1,emotion_speech,02,Female,calm
2,03-01-01-01-01-01-03.wav,2,emotion_speech,02,Male,calm
3,03-01-01-01-01-01-04.wav,3,emotion_speech,02,Female,calm
4,03-01-01-01-01-01-05.wav,4,emotion_speech,02,Male,calm
...,...,...,...,...,...,...
4235,YAF_youth_fear.wav,4235,toronto_speech,06,NaN,fear
4236,YAF_youth_happy.wav,4236,toronto_speech,03,NaN,happy
4237,YAF_youth_neutral.wav,4237,toronto_speech,02,NaN,neutral
4238,YAF_youth_ps.wav,4238,toronto_speech,08,NaN,ps


In [114]:
final_dataset_df.to_csv(r'../data/voice_feature_data.csv', index = True)

In [115]:
train = pd.read_csv('../data/voice_feature_data.csv')
data_dir = ('../data')
train.head()

,Unnamed: 0,file_name,ID,dir,Class,Gender,Emotion
0,0,03-01-01-01-01-01-01.wav,0,emotion_speech,2,Male,calm
1,1,03-01-01-01-01-01-02.wav,1,emotion_speech,2,Female,calm
2,2,03-01-01-01-01-01-03.wav,2,emotion_speech,2,Male,calm
3,3,03-01-01-01-01-01-04.wav,3,emotion_speech,2,Female,calm
4,4,03-01-01-01-01-01-05.wav,4,emotion_speech,2,Male,calm


# CREATE FEATURES

In [116]:
train.Class.value_counts()

2    688
7    592
3    592
6    592
5    592
8    592
4    592
Name: Class, dtype: int64

In [117]:
def parser(row):
   # function to load files and extract features
   file_name = os.path.join(os.path.abspath(data_dir), str(row.dir), str(row.file_name))

   # handle exception to check if there isn't a file which is corrupted
   try:
      # extraction
      X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
      # extract mfcc data
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
   except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None, None
 
   feature = mfccs
   label = row.Class
 
   return [feature, label]

temp = train.apply(parser, axis=1)

In [ ]:
#DataFrame of Features and Labels
audio = pd.DataFrame(temp.tolist(), index= temp.index)
audio.columns = ['feature', 'label']
print(audio)

In [ ]:
#set list as object
new_audio= audio.astype(object)
new_audio

In [ ]:
#Write to CSV
new_audio.to_csv('../data/voice_data.csv')